In [4]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from dotenv import load_dotenv, find_dotenv

from langchain.tools import E2BDataAnalysisTool
import os
from langchain.agents import AgentExecutor
from langchain.agents import tool, initialize_agent, AgentType, Tool
from langchain.chat_models import AzureChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_anthropic.experimental import ChatAnthropicTools
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI

In [5]:
load_dotenv(find_dotenv())

True

In [6]:
e2b_data_analysis_tool = E2BDataAnalysisTool(api_key=os.getenv("E2B_API_KEY"))


@tool
def install_python_package_to_e2b(package_name: str):
    """
    Installs a python package to the E2B environment. This is useful for when you need to install a package that is not already installed in the E2B environment.
    """
    try:
        e2b_data_analysis_tool.install_python_packages(package_name)
        return f"Successfully installed {package_name} to the E2B environment."
    except Exception as e:
        return f"Failed to install {package_name} to the E2B environment. Error: {e}"
    

tools = [e2b_data_analysis_tool, install_python_package_to_e2b]

Exception in thread e2b-sandbox-refresh:
Traceback (most recent call last):
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connectionpool.py", line 793, in urlopen
    response = self._make_request(
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
  File "C:\Python39\lib\http\client.py", line 1349, in getresponse
    response.begin()
  File "C:\Python39\lib\http\client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "C:\Python39\lib\http\client.py", line 277, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Python39\lib\socket.py", line 704, 

# OPENAI Agent

In [7]:


# gpt-4-0125-preview
# gpt-3.5-turbo-0125
# gpt-3.5-turbo-instruct

# Create a chat prompt template
llm = ChatOpenAI(model="gpt-3.5-turbo-instruct", temperature=0,
                 openai_api_key=os.getenv("OPENAI_API_KEY"))

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful air taffic control operator, that can also use tools to answer questions and perform tasks",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [7]:
agent_executor.invoke(
    {"input": question})



> Entering new AgentExecutor chain...

Invoking: `e2b_data_analysis` with `{'python_code': "from geopy.distance import geodesic\nfrom geopy.point import Point\n\n# Aircraft positions in decimal degrees\npos_KL204 = Point('N52 00 0.0', 'E004 00 0.0')\npos_KL420 = Point('N52 00 0.0', 'E003 12 0.0')\n\n# Calculate the midpoint\nmidpoint = geodesic().midpoint(pos_KL204, pos_KL420)\n\n# Distance between KL204 and KL420\ndistance_km = geodesic(pos_KL204, pos_KL420).kilometers\n\n# Speed of both aircraft in km/h (1 knot = 1.852 km/h)\nspeed_kmh = 498 * 1.852\n\n# Time to closest approach point in hours\n# Since both are heading towards each other, we divide the distance by the sum of their speeds\ntime_hours = distance_km / (2 * speed_kmh)\n\n# Convert time to minutes\ntime_minutes = time_hours * 60\n\nprint(f'Midpoint: {midpoint}')\nprint(f'Time to closest approach: {time_minutes:.2f} minutes')"}`
responded: To find the position of the closest approach point between these two aircraft and 

{'input': 'Based on this info what wil be the position of the closest approach point between these aircraft and after how much time will it happen: \n\n \n\'Aircraft ids: [\'KL204\', \'KL420\']\nInfo on KL204 B744 index = 0\nPos: N52\'00\'0.0"  E004\'00\'0.0"\nHdg: 270   Trk: 270\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 350/498/498 kts   M: 0.827\n\nInfo on KL420 B744 index = 1\nPos: N52\'00\'0.0"  E003\'12\'0.0"\nHdg: 090   Trk: 090\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 350/498/498 kts   M: 0.827\n\n\'\n',
 'output': 'The closest approach point between the two aircraft will be at the coordinates **N52°00\'0.0" E003°36\'0.0"**. They will reach this point approximately **1.78 minutes** from now, assuming they maintain their current headings and speeds.',
 'intermediate_steps': [(OpenAIToolAgentAction(tool='e2b_data_analysis', tool_input={'python_code': "from geopy.distance import geodesic\nfrom geopy.point import Point\n\n# Aircraft positions in decimal degrees\npos_KL204 = Point('N52 

# ReACT Anthropic Agent

In [ ]:
# Artifacts are charts created by matplotlib when `plt.show()` is called
def save_artifact(artifact):
    print("New matplotlib chart generated:", artifact.name)
    # Download the artifact as `bytes` and leave it up to the user to display them (on frontend, for example)
    file = artifact.download()
    basename = os.path.basename(artifact.name)

    # Save the chart to the `charts` directory
    with open(f"./charts/{basename}", "wb") as f:
        f.write(file)


e2b_data_analysis_tool = E2BDataAnalysisTool(
    api_key=os.getenv("E2B_API_KEY"),
    on_stdout=lambda stdout: print("stdout:", stdout),
    on_stderr=lambda stderr: print("stderr:", stderr),
    on_artifact=save_artifact,
)

tools = [e2b_data_analysis_tool.as_tool(), install_python_package_to_e2b]



# Choose the LLM to use
#llm = ChatOpenAI(model='gpt-4-0125-preview')
llm = ChatAnthropicTools(model='claude-3-sonnet-20240229',anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"))

#llm = ChatOpenAI()
# Get the prompt to use - you can modify this!
#prompt = hub.pull("hwchase17/react")
# Construct the ReAct agent
agent = initialize_agent(tools=tools, llm=llm, verbose=True,
                         agent=AgentType.OPENAI_FUNCTIONS)
# Create an agent executor by passing in the agent and tools
#agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent.run(question)
#agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...
To find the position of the closest approach point between the two aircraft and the time it will happen, we need to analyze their positions, headings, and speeds.

Given:
- Aircraft KL204 is at position N52°00'0.0" E004°00'0.0", heading 270° (west), and traveling at a ground speed of 498 knots.
- Aircraft KL420 is at position N52°00'0.0" E003°12'0.0", heading 090° (east), and traveling at a ground speed of 498 knots.

To find the closest approach point, we need to determine the intersection point of their flight paths.

Step 1: Convert the positions to decimal degrees.
KL204: N52.0000° E004.0000°
KL420: N52.0000° E003.2000°

Step 2: Calculate the relative distance between the two aircraft's initial positions.
Distance = 48 nautical miles (approximately)

Step 3: Calculate the relative speed of the aircraft towards each other.
Relative speed = 498 knots + 498 knots = 996 knots

Step 4: Calculate the time it takes for the aircraft to reach the clos

'To find the position of the closest approach point between the two aircraft and the time it will happen, we need to analyze their positions, headings, and speeds.\n\nGiven:\n- Aircraft KL204 is at position N52°00\'0.0" E004°00\'0.0", heading 270° (west), and traveling at a ground speed of 498 knots.\n- Aircraft KL420 is at position N52°00\'0.0" E003°12\'0.0", heading 090° (east), and traveling at a ground speed of 498 knots.\n\nTo find the closest approach point, we need to determine the intersection point of their flight paths.\n\nStep 1: Convert the positions to decimal degrees.\nKL204: N52.0000° E004.0000°\nKL420: N52.0000° E003.2000°\n\nStep 2: Calculate the relative distance between the two aircraft\'s initial positions.\nDistance = 48 nautical miles (approximately)\n\nStep 3: Calculate the relative speed of the aircraft towards each other.\nRelative speed = 498 knots + 498 knots = 996 knots\n\nStep 4: Calculate the time it takes for the aircraft to reach the closest approach poi

WebSocket received error while receiving messages: no close frame received or sent
Exception in thread e2b-sandbox-refresh:
Traceback (most recent call last):
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connectionpool.py", line 793, in urlopen
    response = self._make_request(
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\urllib3\connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
  File "C:\Python39\lib\http\client.py", line 1349, in getresponse
    response.begin()
  File "C:\Python39\lib\http\client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "C:\Python39\lib\http\client.py", line 277, in _read_status
    line = str(self.fp.r

In [40]:
from langchain.agents import create_react_agent
from langchain_community.chat_models import ChatAnthropic
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

tools = [PythonREPLTool()]

instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
llm = ChatAnthropic(anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"))
llm = ChatOpenAI(model="gpt-4-0125-preview")
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...
To solve this problem, we can consider it in terms of relative motion between the two aircraft. Given that both aircraft are flying at the same altitude and with the same ground speed but in opposite directions towards each other, the closest approach point will be directly between their starting positions, provided there are no changes in their speeds or directions.

The starting positions of the aircraft are:
- KL204 at longitude E004'00'0.0"
- KL420 at longitude E003'12'0.0"

The distance between the two longitudes needs to be calculated to find the midpoint where they will have their closest approach. Then, using their relative speed, we can calculate the time it will take for them to reach this point.

First, we need to convert the longitude positions from degrees, minutes, and seconds to decimal degrees to make them easier to work with. Then, we can calculate the midpoint and the time taken to reach this point, considering the ground speed 

{'input': 'Based on this info what wil be the position of the closest approach point between these aircraft and after how much time will it happen: \n\n \n\'Aircraft ids: [\'KL204\', \'KL420\']\nInfo on KL204 B744 index = 0\nPos: N52\'00\'0.0"  E004\'00\'0.0"\nHdg: 270   Trk: 270\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 350/498/498 kts   M: 0.827\n\nInfo on KL420 B744 index = 1\nPos: N52\'00\'0.0"  E003\'12\'0.0"\nHdg: 090   Trk: 090\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 350/498/498 kts   M: 0.827\n\n\'\n',
 'output': 'The closest approach point between the aircraft will be at a longitude of E003\'36\'0.0". They will reach this point after approximately 0.048 hours, which is about 2.89 minutes.'}